<a href="https://colab.research.google.com/github/khanghoang2351-design/AI-Final/blob/main/Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os

# Kết nối Google Drive vào Colab
drive.mount('/content/drive')

# --- QUAN TRỌNG: hãy sửa đường dẫn này ---
# Đây là đường dẫn đến thư mục 'dataset_split' trên Drive của .
# Ví dụ: Nếu nó nằm trong 'My Drive/DoAn_AI/dataset_split'
# thì G_DRIVE_PATH = '/content/drive/MyDrive/DoAn_AI'
#
# Nếu nó nằm ngay trong 'My Drive/dataset_split', thì:
G_DRIVE_PATH = '/content/drive/MyDrive'
# -------------------------------------------------

# Xây dựng các đường dẫn tới thư mục train, val, test
BASE_PATH = os.path.join(G_DRIVE_PATH, 'dataset_split')
TRAIN_PATH = os.path.join(BASE_PATH, 'train')
VAL_PATH = os.path.join(BASE_PATH, 'val')
TEST_PATH = os.path.join(BASE_PATH, 'test')

# In ra để kiểm tra
print(f"Đường dẫn thư mục Train: {TRAIN_PATH}")
print(f"Đường dẫn thư mục Val: {VAL_PATH}")
print(f"Đường dẫn thư mục Test: {TEST_PATH}")

# Kiểm tra xem Colab có "nhìn thấy" các thư mục con (các món ăn) không
print("\n--- Các thư mục trong 'train': ---")
!ls "$TRAIN_PATH"


Mounted at /content/drive
Đường dẫn thư mục Train: /content/drive/MyDrive/dataset_split/train
Đường dẫn thư mục Val: /content/drive/MyDrive/dataset_split/val
Đường dẫn thư mục Test: /content/drive/MyDrive/dataset_split/test

--- Các thư mục trong 'train': ---
'Cá hú kho'	      'Canh rau'       'Rau xào'     'Thịt kho trứng'
'canh chua có cá'     'Cơm trắng'      'Sườn nướng'  'Trứng chiên'
'canh chua không cá'  'Đậu hủ sốt cà'  'Thịt kho'


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# --- Các tham số cho mô hình ---

# MobileNetV2 (mô hình chúng ta sẽ dùng) yêu cầu ảnh đầu vào 224x224
IMG_SIZE = 224
# Tải ảnh theo từng lô 32 ảnh
BATCH_SIZE = 32
# Số lượng lớp (món ăn)
NUM_CLASSES = 11

# --- 1. Tạo công cụ Tăng cường dữ liệu (cho tập Train) ---
# Chúng ta chỉ tăng cường tập train để mô hình học được nhiều biến thể
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, # Chuẩn hóa ảnh theo yêu cầu của MobileNetV2
    rotation_range=30,      # Xoay ngẫu nhiên ảnh đến 30 độ
    width_shift_range=0.2,  # Dịch chuyển chiều ngang
    height_shift_range=0.2, # Dịch chuyển chiều dọc
    shear_range=0.2,        # Bóp méo ảnh
    zoom_range=0.2,         # Phóng to/thu nhỏ ngẫu nhiên
    horizontal_flip=True,   # Lật ngang ảnh
    fill_mode='nearest'     # Lấp đầy pixel bị thiếu khi xoay/dịch
)

# --- 2. Tạo công cụ cho tập Validation và Test ---
# Chúng ta KHÔNG tăng cường tập val/test, chỉ chuẩn hóa chúng
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# --- 3. Tạo các "dòng chảy" (generators) dữ liệu từ thư mục ---
# Các generator này sẽ tự động đọc ảnh từ thư mục và đưa vào mô hình

train_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,                     # Đường dẫn đã xác nhận ở Bước 1
    target_size=(IMG_SIZE, IMG_SIZE), # Thay đổi kích thước ảnh về 224x224
    batch_size=BATCH_SIZE,
    class_mode='categorical'        # Vì đây là bài toán phân loại nhiều lớp
)

validation_generator = validation_datagen.flow_from_directory(
    VAL_PATH,                       # Đường dẫn đã xác nhận ở Bước 1
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False                   # Không cần xáo trộn tập validation
)

test_generator = test_datagen.flow_from_directory(
    TEST_PATH,                      # Đường dẫn đã xác nhận ở Bước 1
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False                   # Không cần xáo trộn tập test
)

# In ra kết quả để xác nhận
print("\n--- Đã tạo xong Data Generators ---")
print(f"Tổng số ảnh trong tập Train: {train_generator.samples}")
print(f"Tổng số ảnh trong tập Validation: {validation_generator.samples}")
print(f"Tổng số ảnh trong tập Test: {test_generator.samples}")
print("\nCác lớp (món ăn) được tìm thấy:")
print(train_generator.class_indices)

Found 4866 images belonging to 11 classes.
Found 1388 images belonging to 11 classes.
Found 703 images belonging to 11 classes.

--- Đã tạo xong Data Generators ---
Tổng số ảnh trong tập Train: 4866
Tổng số ảnh trong tập Validation: 1388
Tổng số ảnh trong tập Test: 703

Các lớp (món ăn) được tìm thấy:
{'Canh rau': 0, 'Cá hú kho': 1, 'Cơm trắng': 2, 'Rau xào': 3, 'Sườn nướng': 4, 'Thịt kho': 5, 'Thịt kho trứng': 6, 'Trứng chiên': 7, 'canh chua có cá': 8, 'canh chua không cá': 9, 'Đậu hủ sốt cà': 10}


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# --- 1. Tải mô hình gốc (Base Model) ---

# Tải MobileNetV2, bỏ lớp phân loại cuối (include_top=False)
# và sử dụng trọng số đã huấn luyện trên 'imagenet'
base_model = MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3), # Kích thước ảnh 224x224, 3 kênh màu
    include_top=False,
    weights='imagenet'
)

# --- 2. Đóng băng mô hình gốc ---
# Chúng ta không muốn huấn luyện lại các lớp này, chỉ dùng chúng để trích xuất đặc trưng
base_model.trainable = False

# --- 3. Xây dựng lớp phân loại mới (Our Classifier Head) ---

# Lấy đầu ra của mô hình gốc
x = base_model.output

# Thêm một lớp "gộp" (pooling) để giảm số lượng tham số
x = GlobalAveragePooling2D()(x)

# Thêm một lớp Dropout để giảm "học vẹt" (overfitting)
# Nó sẽ "tắt" ngẫu nhiên 50% nơ-ron trong quá trình huấn luyện
x = Dropout(0.5)(x)

# Lớp Dense cuối cùng: Lớp quyết định!
# Có NUM_CLASSES (11) nơ-ron, và dùng 'softmax'
# để tính xác suất cho từng món ăn
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# --- 4. Kết hợp thành mô hình hoàn chỉnh ---
# Đầu vào là đầu vào của base_model
# Đầu ra là lớp 'predictions' chúng ta vừa tạo
model = Model(inputs=base_model.input, outputs=predictions)

# --- 5. Biên dịch (Compile) mô hình ---
# Chúng ta dùng 'Adam' là một trình tối ưu (optimizer) hiệu quả
# 'categorical_crossentropy' là hàm mất mát (loss) chuẩn cho phân loại đa lớp
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# --- 6. In cấu trúc mô hình ---
print("--- Cấu trúc mô hình ---")
model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
--- Cấu trúc mô hình ---


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,272,075 (8.67 MB)

 Trainable params: 14,091 (55.04 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# --- 1. Thiết lập Checkpoint để lưu mô hình tốt nhất ---

# Tên file để lưu mô hình
# Chúng ta sẽ lưu nó vào thư mục G_DRIVE_PATH (đã xác định ở Bước 1)
checkpoint_path = os.path.join(G_DRIVE_PATH, "food_classifier_best.h5")

# Tạo callback:
# - 'val_accuracy': Theo dõi độ chính xác trên tập validation
# - save_best_only=True: Chỉ lưu khi tìm thấy mô hình "tốt nhất"
# - mode='max': "Tốt nhất" nghĩa là 'val_accuracy' cao nhất
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1 # Báo cáo khi lưu mô hình
)

# --- 2. Xác định số Epochs (số lượt học) ---
# 20-30 epochs thường là đủ cho giai đoạn đầu của Transfer Learning
EPOCHS = 25

# --- 3. Bắt đầu Huấn luyện ---
print("--- Bắt đầu Huấn luyện Giai đoạn 1 (Đóng băng) ---")

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=[checkpoint_callback] # Thêm callback vào quá trình huấn luyện
)

print("\n--- Huấn luyện Giai đoạn 1 Hoàn tất ---")
print(f"Mô hình tốt nhất đã được lưu tại: {checkpoint_path}")

--- Bắt đầu Huấn luyện Giai đoạn 1 (Đóng băng) ---


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 39s/step - accuracy: 0.4890 - loss: 1.5674 
Epoch 1: val_accuracy improved from -inf to 0.87248, saving model to /content/drive/MyDrive/food_classifier_best.h5


153/153 ━━━━━━━━━━━━━━━━━━━━ 7679s 50s/step - accuracy: 0.4901 - loss: 1.5639 - val_accuracy: 0.8725 - val_loss: 0.3756
Epoch 2/25
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 540ms/step - accuracy: 0.8285 - loss: 0.4916
Epoch 2: val_accuracy improved from 0.87248 to 0.90778, saving model to /content/drive/MyDrive/food_classifier_best.h5


153/153 ━━━━━━━━━━━━━━━━━━━━ 94s 615ms/step - accuracy: 0.8286 - loss: 0.4914 - val_accuracy: 0.9078 - val_loss: 0.2691
Epoch 3/25
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 537ms/step - accuracy: 0.8627 - loss: 0.3872
Epoch 3: val_accuracy did not improve from 0.90778
153/153 ━━━━━━━━━━━━━━━━━━━━ 91s 596ms/step - accuracy: 0.8628 - loss: 0.3870 - val_accuracy: 0.9056 - val_loss: 0.2441
Epoch 4/25
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 538ms/step - accuracy: 0.8867 - loss: 0.3262
Epoch 4: val_accuracy improved from 0.90778 to 0.91931, saving model to /content/drive/MyDrive/food_classifier_best.h5


...

153/153 ━━━━━━━━━━━━━━━━━━━━ 94s 613ms/step - accuracy: 0.9213 - loss: 0.2240 - val_accuracy: 0.9452 - val_loss: 0.1479
Epoch 10/25
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 547ms/step - accuracy: 0.9162 - loss: 0.2334
Epoch 10: val_accuracy did not improve from 0.94524
153/153 ━━━━━━━━━━━━━━━━━━━━ 92s 601ms/step - accuracy: 0.9162 - loss: 0.2334 - val_accuracy: 0.9388 - val_loss: 0.1606
Epoch 11/25
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 545ms/step - accuracy: 0.9330 - loss: 0.2012
Epoch 11: val_accuracy improved from 0.94524 to 0.94741, saving model to /content/drive/MyDrive/food_classifier_best.h5


153/153 ━━━━━━━━━━━━━━━━━━━━ 94s 618ms/step - accuracy: 0.9329 - loss: 0.2013 - val_accuracy: 0.9474 - val_loss: 0.1364
Epoch 12/25
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 544ms/step - accuracy: 0.9242 - loss: 0.1930
Epoch 12: val_accuracy improved from 0.94741 to 0.94813, saving model to /content/drive/MyDrive/food_classifier_best.h5


153/153 ━━━━━━━━━━━━━━━━━━━━ 142s 615ms/step - accuracy: 0.9242 - loss: 0.1931 - val_accuracy: 0.9481 - val_loss: 0.1281
Epoch 13/25
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 536ms/step - accuracy: 0.9271 - loss: 0.1998
Epoch 13: val_accuracy did not improve from 0.94813
153/153 ━━━━━━━━━━━━━━━━━━━━ 91s 593ms/step - accuracy: 0.9271 - loss: 0.1999 - val_accuracy: 0.9452 - val_loss: 0.1483
Epoch 14/25
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 538ms/step - accuracy: 0.9240 - loss: 0.2087
Epoch 14: val_accuracy improved from 0.94813 to 0.95605, saving model to /content/drive/MyDrive/food_classifier_best.h5


153/153 ━━━━━━━━━━━━━━━━━━━━ 93s 608ms/step - accuracy: 0.9240 - loss: 0.2087 - val_accuracy: 0.9561 - val_loss: 0.1255
Epoch 15/25
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 540ms/step - accuracy: 0.9311 - loss: 0.1940
Epoch 15: val_accuracy did not improve from 0.95605
153/153 ━━━━━━━━━━━━━━━━━━━━ 92s 600ms/step - accuracy: 0.9311 - loss: 0.1940 - val_accuracy: 0.9517 - val_loss: 0.1198
Epoch 16/25
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 539ms/step - accuracy: 0.9355 - loss: 0.1983
Epoch 16: val_accuracy did not improve from 0.95605
153/153 ━━━━━━━━━━━━━━━━━━━━ 91s 595ms/step - accuracy: 0.9355 - loss: 0.1982 - val_accuracy: 0.9517 - val_loss: 0.1219
Epoch 17/25
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 536ms/step - accuracy: 0.9264 - loss: 0.1992
Epoch 17: val_accuracy did not improve from 0.95605
153/153 ━━━━━━━━━━━━━━━━━━━━ 142s 593ms/step - accuracy: 0.9264 - loss: 0.1992 - val_accuracy: 0.9474 - val_loss: 0.1338
Epoch 18/25
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 544ms/step - accuracy: 0.9304 - loss: 0.1950
Epoch 1

153/153 ━━━━━━━━━━━━━━━━━━━━ 93s 607ms/step - accuracy: 0.9335 - loss: 0.1830 - val_accuracy: 0.9589 - val_loss: 0.1187
Epoch 21/25
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 540ms/step - accuracy: 0.9307 - loss: 0.1887
Epoch 21: val_accuracy did not improve from 0.95893
153/153 ━━━━━━━━━━━━━━━━━━━━ 91s 598ms/step - accuracy: 0.9307 - loss: 0.1887 - val_accuracy: 0.9546 - val_loss: 0.1182
Epoch 22/25
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 543ms/step - accuracy: 0.9362 - loss: 0.1811
Epoch 22: val_accuracy did not improve from 0.95893
153/153 ━━━━━━━━━━━━━━━━━━━━ 92s 602ms/step - accuracy: 0.9362 - loss: 0.1811 - val_accuracy: 0.9575 - val_loss: 0.1133
Epoch 23/25
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 543ms/step - accuracy: 0.9366 - loss: 0.1686
Epoch 23: val_accuracy did not improve from 0.95893
153/153 ━━━━━━━━━━━━━━━━━━━━ 92s 602ms/step - accuracy: 0.9365 - loss: 0.1687 - val_accuracy: 0.9431 - val_loss: 0.1338
Epoch 24/25
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 552ms/step - accuracy: 0.9283 - loss: 0.1921
Epoch 24